In [47]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from transformers import AdamW
from tqdm import tqdm

In [48]:
language="it"
modelname="mBert"
input = "gpt4"
test = "Dodiom"

In [49]:
if input == "gpt4":
  gpt4_flag=True
else:
  gpt4_flag=False

In [50]:
# JP encoding='euc-jp' other encoding normal CHECK ALWAYS DATASETS
df_train = pd.read_csv(f'train_gpt4_it.csv')
df_eval = pd.read_csv(f'eval_gpt4_it.csv')
df_test = pd.read_csv(f'dodiom_it_test.csv')

In [51]:
def get_category_mapping(language):
    if language in ['en', 'jp']:
        category_mapping = {
            'figurative': 'idiom',
            'literal': 'nonidiom'
        }
    elif language == 'tr':
        category_mapping = {
            'mecaz': 'idiom',
            'gerçek': 'nonidiom'
        }
    elif language == 'it':
        category_mapping = {
            'figurato': 'idiom',
            'letterale': 'nonidiom'
        }
    else:
        raise ValueError(f"Language '{language}' is not supported.")

    return category_mapping

In [52]:
category_mapping = get_category_mapping(language)
print(category_mapping)

{'figurato': 'idiom', 'letterale': 'nonidiom'}


In [53]:
if gpt4_flag or language in ['en', 'jp']:
  df_train['category'] = df_train['category'].map(category_mapping)
df_train

,submission,category,idiom,type
0,Possiamo davvero prendere con le pinze questa ...,idiom,Prendere con le pinze,Enhanced-prompting
1,"Anche di fronte alle difficoltà estreme, getta...",idiom,Gettare la spugna,Enhanced-prompting
2,"In una discussione seria, chi ha il becco lung...",idiom,Avere il becco lungo,Zero-shot
3,Le promesse di cambiamenti rapidi possono suon...,idiom,Prendere con le pinze,Zero-shot
4,Non ho preso interamente con le pinze le notiz...,idiom,Prendere con le pinze,Enhanced-prompting
...,...,...,...,...
5915,Nel tentativo di recuperare l’oggetto caduto d...,nonidiom,Far leva,Zero-shot
5916,Dopo il temporale che aveva oscurato lituttta ...,nonidiom,Brancolare nel buio,Zero-shot
5917,"Al festival, il giocoliere ha deciso di buttar...",nonidiom,Buttare al vento,Enhanced-prompting
5918,"Spesso, le scuole insegnano ai bambini come co...",nonidiom,Coltivare il proprio orto,Enhanced-prompting


In [54]:
if gpt4_flag or language in ['en', 'jp']:
  df_eval['category'] = df_eval['category'].map(category_mapping)
df_eval

,submission,category,idiom,type
0,Carmen non aveva la corda al collo nonostante ...,idiom,Aver la corda al collo,Enhanced-prompting
1,L'insegnante ha fatto leva sulla curiosità inn...,idiom,Far leva,Zero-shot
2,"Nel contesto lavorativo, talvolta è più produt...",idiom,Coltivare il proprio orto,Zero-shot
3,Avendo compreso finalmente quella teoria compl...,idiom,Essere a cavallo,Zero-shot
4,La campagna ha fatto leva sulla solidarietà co...,idiom,Far leva,Zero-shot
...,...,...,...,...
1475,Carlo e Sofia andarono in barca a remi al parc...,nonidiom,Andare in barca,Zero-shot
1476,Ho visto un cappello unico che aveva la coda d...,nonidiom,Aver la coda di paglia,Enhanced-prompting
1477,Mentre cercava l'interruttore nel nuovo appart...,nonidiom,Brancolare nel buio,Zero-shot
1478,"Mentre sistemavo il garage, ho gettato la spug...",nonidiom,Gettare la spugna,Zero-shot


In [55]:
if language in ['en', 'jp']:
  df_test['category'] = df_test['category'].map(category_mapping)

In [56]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Abbaiare alla luna o roteare gatti non può cur...,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaiare', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'o', 'roteare', '...","['Abbaiare', 'alla', 'luna', 'o', 'roteare', '..."
1,abbaiare alla luna,Non è possibile che tu debba abbaiare per tutt...,idiom,0.500000,1,1,0,"[6, 9, 10]","['abbaiare', 'alla', 'luna']","['non', 'essere', 'possibile', 'che', 'tu', 'd...","['Non', 'è', 'possibile', 'che', 'tu', 'debba'..."
2,abbaiare alla luna,Chiedere alla prof di pubblicare le slide è co...,idiom,0.500000,1,1,0,"[9, 10, 11]","['abbaiare', 'alla', 'luna']","['chiedere', 'alla', 'prof', 'di', 'pubblicare...","['Chiedere', 'alla', 'prof', 'di', 'pubblicare..."
3,abbaiare alla luna,Abbai almeno 10 volte al giorno alla luna,idiom,0.000000,0,2,0,"[0, 6, 7]","['Abbai', 'alla', 'luna']","['abbaiare', 'almeno', '10', 'volta', 'al', 'g...","['Abbai', 'almeno', '10', 'volte', 'al', 'gior..."
4,abbaiare alla luna,Sprechi troppo tempo ad abbaiare alla luna,idiom,0.666667,2,1,0,"[4, 5, 6]","['abbaiare', 'alla', 'luna']","['sprecare', 'troppo', 'tempo', 'ad', 'abbaiar...","['Sprechi', 'troppo', 'tempo', 'ad', 'abbaiare..."
...,...,...,...,...,...,...,...,...,...,...,...
2279,voltare la faccia,"Puoi voltare la faccia per favore, voglio fart...",nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['potere', 'voltare', 'la', 'faccia', 'per', '...","['Puoi', 'voltare', 'la', 'faccia', 'per', 'fa..."
2280,voltare la faccia,per favore puoi voltare la faccia verso di me ...,nonidiom,1.000000,3,0,0,"[3, 5]","['voltare', 'faccia']","['per', 'favore', 'potere', 'voltare', 'la', '...","['per', 'favore', 'puoi', 'voltare', 'la', 'fa..."
2281,voltare la faccia,devi voltare la faccia da entrambi i lati per ...,nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['deviare', 'voltare', 'la', 'faccia', 'da', '...","['devi', 'voltare', 'la', 'faccia', 'da', 'ent..."
2282,voltare la faccia,Appena mi ha visto ha voltato la faccia,nonidiom,0.500000,1,1,0,"[5, 7]","['voltato', 'faccia']","['appena', 'mi', 'avere', 'vistare', 'avere', ...","['Appena', 'mi', 'ha', 'visto', 'ha', 'voltato..."


In [57]:
df_train['category'] = df_train['category'].str.lower()
df_eval['category'] = df_eval['category'].str.lower()
df_test['category'] = df_test['category'].str.lower()

In [58]:
df_train.head()

,submission,category,idiom,type
0,Possiamo davvero prendere con le pinze questa ...,idiom,Prendere con le pinze,Enhanced-prompting
1,"Anche di fronte alle difficoltà estreme, getta...",idiom,Gettare la spugna,Enhanced-prompting
2,"In una discussione seria, chi ha il becco lung...",idiom,Avere il becco lungo,Zero-shot
3,Le promesse di cambiamenti rapidi possono suon...,idiom,Prendere con le pinze,Zero-shot
4,Non ho preso interamente con le pinze le notiz...,idiom,Prendere con le pinze,Enhanced-prompting


In [59]:
df_eval.head()

,submission,category,idiom,type
0,Carmen non aveva la corda al collo nonostante ...,idiom,Aver la corda al collo,Enhanced-prompting
1,L'insegnante ha fatto leva sulla curiosità inn...,idiom,Far leva,Zero-shot
2,"Nel contesto lavorativo, talvolta è più produt...",idiom,Coltivare il proprio orto,Zero-shot
3,Avendo compreso finalmente quella teoria compl...,idiom,Essere a cavallo,Zero-shot
4,La campagna ha fatto leva sulla solidarietà co...,idiom,Far leva,Zero-shot


In [60]:
df_test.head()

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Abbaiare alla luna o roteare gatti non può cur...,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaiare', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'o', 'roteare', '...","['Abbaiare', 'alla', 'luna', 'o', 'roteare', '..."
1,abbaiare alla luna,Non è possibile che tu debba abbaiare per tutt...,idiom,0.500000,1,1,0,"[6, 9, 10]","['abbaiare', 'alla', 'luna']","['non', 'essere', 'possibile', 'che', 'tu', 'd...","['Non', 'è', 'possibile', 'che', 'tu', 'debba'..."
2,abbaiare alla luna,Chiedere alla prof di pubblicare le slide è co...,idiom,0.500000,1,1,0,"[9, 10, 11]","['abbaiare', 'alla', 'luna']","['chiedere', 'alla', 'prof', 'di', 'pubblicare...","['Chiedere', 'alla', 'prof', 'di', 'pubblicare..."
3,abbaiare alla luna,Abbai almeno 10 volte al giorno alla luna,idiom,0.000000,0,2,0,"[0, 6, 7]","['Abbai', 'alla', 'luna']","['abbaiare', 'almeno', '10', 'volta', 'al', 'g...","['Abbai', 'almeno', '10', 'volte', 'al', 'gior..."
4,abbaiare alla luna,Sprechi troppo tempo ad abbaiare alla luna,idiom,0.666667,2,1,0,"[4, 5, 6]","['abbaiare', 'alla', 'luna']","['sprecare', 'troppo', 'tempo', 'ad', 'abbaiar...","['Sprechi', 'troppo', 'tempo', 'ad', 'abbaiare..."


In [61]:
# Encode the 'category' column
label_encoder = LabelEncoder()
df_train['category_encoded'] = label_encoder.fit_transform(df_train['category'])
df_eval['category_encoded'] = label_encoder.transform(df_eval['category'])
df_test['category_encoded'] = label_encoder.transform(df_test['category'])

In [62]:
df_train

,submission,category,idiom,type,category_encoded
0,Possiamo davvero prendere con le pinze questa ...,idiom,Prendere con le pinze,Enhanced-prompting,0
1,"Anche di fronte alle difficoltà estreme, getta...",idiom,Gettare la spugna,Enhanced-prompting,0
2,"In una discussione seria, chi ha il becco lung...",idiom,Avere il becco lungo,Zero-shot,0
3,Le promesse di cambiamenti rapidi possono suon...,idiom,Prendere con le pinze,Zero-shot,0
4,Non ho preso interamente con le pinze le notiz...,idiom,Prendere con le pinze,Enhanced-prompting,0
...,...,...,...,...,...
5915,Nel tentativo di recuperare l’oggetto caduto d...,nonidiom,Far leva,Zero-shot,1
5916,Dopo il temporale che aveva oscurato lituttta ...,nonidiom,Brancolare nel buio,Zero-shot,1
5917,"Al festival, il giocoliere ha deciso di buttar...",nonidiom,Buttare al vento,Enhanced-prompting,1
5918,"Spesso, le scuole insegnano ai bambini come co...",nonidiom,Coltivare il proprio orto,Enhanced-prompting,1


In [63]:
df_eval

,submission,category,idiom,type,category_encoded
0,Carmen non aveva la corda al collo nonostante ...,idiom,Aver la corda al collo,Enhanced-prompting,0
1,L'insegnante ha fatto leva sulla curiosità inn...,idiom,Far leva,Zero-shot,0
2,"Nel contesto lavorativo, talvolta è più produt...",idiom,Coltivare il proprio orto,Zero-shot,0
3,Avendo compreso finalmente quella teoria compl...,idiom,Essere a cavallo,Zero-shot,0
4,La campagna ha fatto leva sulla solidarietà co...,idiom,Far leva,Zero-shot,0
...,...,...,...,...,...
1475,Carlo e Sofia andarono in barca a remi al parc...,nonidiom,Andare in barca,Zero-shot,1
1476,Ho visto un cappello unico che aveva la coda d...,nonidiom,Aver la coda di paglia,Enhanced-prompting,1
1477,Mentre cercava l'interruttore nel nuovo appart...,nonidiom,Brancolare nel buio,Zero-shot,1
1478,"Mentre sistemavo il garage, ho gettato la spug...",nonidiom,Gettare la spugna,Zero-shot,1


In [64]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words,category_encoded
0,abbaiare alla luna,Abbaiare alla luna o roteare gatti non può cur...,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaiare', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'o', 'roteare', '...","['Abbaiare', 'alla', 'luna', 'o', 'roteare', '...",0
1,abbaiare alla luna,Non è possibile che tu debba abbaiare per tutt...,idiom,0.500000,1,1,0,"[6, 9, 10]","['abbaiare', 'alla', 'luna']","['non', 'essere', 'possibile', 'che', 'tu', 'd...","['Non', 'è', 'possibile', 'che', 'tu', 'debba'...",0
2,abbaiare alla luna,Chiedere alla prof di pubblicare le slide è co...,idiom,0.500000,1,1,0,"[9, 10, 11]","['abbaiare', 'alla', 'luna']","['chiedere', 'alla', 'prof', 'di', 'pubblicare...","['Chiedere', 'alla', 'prof', 'di', 'pubblicare...",0
3,abbaiare alla luna,Abbai almeno 10 volte al giorno alla luna,idiom,0.000000,0,2,0,"[0, 6, 7]","['Abbai', 'alla', 'luna']","['abbaiare', 'almeno', '10', 'volta', 'al', 'g...","['Abbai', 'almeno', '10', 'volte', 'al', 'gior...",0
4,abbaiare alla luna,Sprechi troppo tempo ad abbaiare alla luna,idiom,0.666667,2,1,0,"[4, 5, 6]","['abbaiare', 'alla', 'luna']","['sprecare', 'troppo', 'tempo', 'ad', 'abbaiar...","['Sprechi', 'troppo', 'tempo', 'ad', 'abbaiare...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
2279,voltare la faccia,"Puoi voltare la faccia per favore, voglio fart...",nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['potere', 'voltare', 'la', 'faccia', 'per', '...","['Puoi', 'voltare', 'la', 'faccia', 'per', 'fa...",1
2280,voltare la faccia,per favore puoi voltare la faccia verso di me ...,nonidiom,1.000000,3,0,0,"[3, 5]","['voltare', 'faccia']","['per', 'favore', 'potere', 'voltare', 'la', '...","['per', 'favore', 'puoi', 'voltare', 'la', 'fa...",1
2281,voltare la faccia,devi voltare la faccia da entrambi i lati per ...,nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['deviare', 'voltare', 'la', 'faccia', 'da', '...","['devi', 'voltare', 'la', 'faccia', 'da', 'ent...",1
2282,voltare la faccia,Appena mi ha visto ha voltato la faccia,nonidiom,0.500000,1,1,0,"[5, 7]","['voltato', 'faccia']","['appena', 'mi', 'avere', 'vistare', 'avere', ...","['Appena', 'mi', 'ha', 'visto', 'ha', 'voltato...",1


In [65]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [66]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenize the data
train_encodings = tokenizer(list(df_train['submission']), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(list(df_eval['submission']), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(list(df_test['submission']), truncation=True, padding=True, return_tensors="pt")
# Convert to torch tensors
train_labels = torch.tensor(df_train['category_encoded'].values)
val_labels = torch.tensor(df_eval['category_encoded'].values)
test_labels = torch.tensor(df_test['category_encoded'].values)

# Prepare to datasets
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=num_labels)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-6)

epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Average loss across all batches
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Evaluation step
    model.eval()
    total_eval_accuracy = 0
    for batch in tqdm(val_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == batch['labels']).cpu().numpy().mean() * 100
        total_eval_accuracy += accuracy

    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Accuracy: {avg_val_accuracy:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/740 [00:00<?, ?it/s]<ipython-input-65-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-65-845b27163e62>:8: User

Epoch 1/4 | Train Loss: 0.22012607337076318


100%|██████████| 185/185 [00:03<00:00, 48.40it/s]


Epoch 1/4 | Validation Accuracy: 94.73%


100%|██████████| 740/740 [01:39<00:00,  7.45it/s]


...

100%|██████████| 740/740 [01:39<00:00,  7.46it/s]


Epoch 3/4 | Train Loss: 0.04136720078059699


100%|██████████| 185/185 [00:03<00:00, 48.96it/s]


Epoch 3/4 | Validation Accuracy: 97.64%


  4%|▎         | 27/740 [00:03<01:36,  7.43it/s]

In [ ]:
# Ensure the model is in evaluation mode
model.eval()

# Move the model to the appropriate device
model.to(device)

# Initialize variables to track accuracy
total_correct = 0
total_predictions = 0

# Store true labels and predictions for F1 score calculation
all_labels = []
all_predictions = []

# No gradient updates needed for evaluation
with torch.no_grad():
    for batch in test_loader:
        # Move batch to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Update tracking variables
        total_correct += (predictions == batch['labels']).sum().item()
        total_predictions += predictions.size(0)

        # Store predictions and true labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Calculate accuracy
test_accuracy = total_correct / total_predictions
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate F1 scores
f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
f1_macro = f1_score(all_labels, all_predictions, average='macro')

print(f'Weighted F1 Score: {f1_weighted:.4f}')
print(f'Macro F1 Score: {f1_macro:.4f}')


In [ ]:
# Generate classification report
report = classification_report(all_labels, all_predictions, digits=4)
print(f'Classification Report for {modelname}, language {language.upper()}, Train {input.upper()}, Test {test.upper()}:')
print(report)